In [3]:
# X-Sec Framework


In [4]:
# init qwen model config
from dotenv import load_dotenv, find_dotenv
from langchain_community.chat_models import ChatTongyi

_ = load_dotenv(find_dotenv("./env/.env"))

llm_model = "qwen-max"

llm = ChatTongyi(temperature=0.8, model=llm_model)

In [5]:
# prepare ttp datas
#!python3.10 ./extract_groups.py
#!pip install tabulate
#!pip install pandas
#!pip install jinja2

import pandas as pd
from tabulate import tabulate
from mitreattack.stix20 import MitreAttackData

# example group id = intrusion-set--fd66436e-4d33-450e-ac4c-f7810f1c85f4
group_id = "intrusion-set--fd66436e-4d33-450e-ac4c-f7810f1c85f4"
selected_group_alias = "FIN13"
mitre_attack_data = MitreAttackData("./mitre-datas/enterprise-attack-15.1.json")
# Define techniques_df as an empty dataframe
techniques_df = pd.DataFrame()
# Get all techniques used by the group
techniques = mitre_attack_data.get_techniques_used_by_group(group_id)
# Check if there are any techniques for the group
if not techniques:
    raise ValueError(f"No techniques found for group {group_id}")
else:
    # Update techniques_df with the techniques
    techniques_df = pd.DataFrame(techniques)
# Create a copy of the DataFrame for generating the LLM prompt
# Add a 'Technique Name' column to both DataFrames
techniques_df["Technique Name"] = techniques_df["object"].apply(lambda x: x["name"])
# Add a 'ATT&CK ID' column to both DataFrames
techniques_df["ATT&CK ID"] = techniques_df["object"].apply(
    lambda x: mitre_attack_data.get_attack_id(x["id"])
)
# Add a 'Phase Name' column to both DataFrames
techniques_df["Phase Name"] = techniques_df["object"].apply(
    lambda x: x["kill_chain_phases"][0]["phase_name"]
)
# Drop duplicate techniques based on Phase Name, Technique Name and ATT&CK ID
techniques_df = techniques_df.drop_duplicates(
    ["Phase Name", "Technique Name", "ATT&CK ID"]
)

selected_techniques_df = (
    techniques_df.groupby("Phase Name", observed=False)[
        ["Phase Name", "Technique Name", "ATT&CK ID"]
    ]
    .apply(lambda x: x.sample(n=1) if len(x) > 0 else None)
    .reset_index(drop=True)
)

# print(
#    tabulate(
#        techniques_df[["Phase Name", "Technique Name", "ATT&CK ID"]],
#        headers=["Phase Name", "Technique Name", "ATT&CK ID"],
#        tablefmt="psql",
#    )
# )

# Create an empty list to hold the kill chain information
kill_chain = []
# Iterate over the rows in the DataFrame
for index, row in selected_techniques_df.iterrows():
    # Extract the phase and technique information
    phase_name = row['Phase Name']
    technique_name = row['Technique Name']
    attack_id = row['ATT&CK ID']
    # Add a string with this information to the kill chain list
    kill_chain.append(f"{phase_name}: {technique_name} ({attack_id})")
# Convert the kill chain list to a string with newline characters between each item
kill_chain_string = "\n".join(["- "+kc for kc in kill_chain])

print(kill_chain_string)

- collection: Data from Local System (T1005)
- command-and-control: Internal Proxy (T1090.001)
- credential-access: LSASS Memory (T1003.001)
- defense-evasion: Make and Impersonate Token (T1134.003)
- discovery: Domain Account (T1087.002)
- execution: Scheduled Task (T1053.005)
- impact: Financial Theft (T1657)
- initial-access: Exploit Public-Facing Application (T1190)
- lateral-movement: Windows Remote Management (T1021.006)
- persistence: Web Shell (T1505.003)
- reconnaissance: Gather Victim Identity Information (T1589)
- resource-development: Tool (T1588.002)


In [6]:
industry_list = [
    "Aerospace / Defense",
    "Agriculture / Food Services",
    "Automotive",
    "Construction",
    "Education",
    "Energy / Utilities",
    "Finance / Banking",
    "Government / Public Sector",
    "Healthcare",
    "Hospitality / Tourism",
    "Insurance",
    "Legal Services",
    "Manufacturing",
    "Media / Entertainment",
    "Non-profit",
    "Real Estate",
    "Retail / E-commerce",
    "Technology / IT",
    "Telecommunication",
    "Transportation / Logistics",
]

industry = "Technology / IT"

company_list = [
    "Small (1-50 employees)",
    "Medium (51-200 employees)",
    "Large (201-1,000 employees)",
    "Enterprise (1,001-10,000 employees)",
    "Large Enterprise (10,000+ employees)",
]
company_size = "Large Enterprise (10,000+ employees)"

cve_2024_5535 = """Issue summary: Calling the OpenSSL API function SSL_select_next_proto with an
empty supported client protocols buffer may cause a crash or memory contents to
be sent to the peer.
Impact summary: A buffer overread can have a range of potential consequences
such as unexpected application beahviour or a crash. In particular this issue
could result in up to 255 bytes of arbitrary private data from memory being sent
to the peer leading to a loss of confidentiality. However, only applications
that directly call the SSL_select_next_proto function with a 0 length list of
supported client protocols are affected by this issue. This would normally never
be a valid scenario and is typically not under attacker control but may occur by
accident in the case of a configuration or programming error in the calling
application.
The OpenSSL API function SSL_select_next_proto is typically used by TLS
applications that support ALPN (Application Layer Protocol Negotiation) or NPN
(Next Protocol Negotiation). NPN is older, was never standardised and
is deprecated in favour of ALPN. We believe that ALPN is significantly more
widely deployed than NPN. The SSL_select_next_proto function accepts a list of
protocols from the server and a list of protocols from the client and returns
the first protocol that appears in the server list that also appears in the
client list. In the case of no overlap between the two lists it returns the
first item in the client list. In either case it will signal whether an overlap
between the two lists was found. In the case where SSL_select_next_proto is
called with a zero length client list it fails to notice this condition and
returns the memory immediately following the client list pointer (and reports
that there was no overlap in the lists).
This function is typically called from a server side application callback for
ALPN or a client side application callback for NPN. In the case of ALPN the list
of protocols supplied by the client is guaranteed by libssl to never be zero in
length. The list of server protocols comes from the application and should never
normally be expected to be of zero length. In this case if the
SSL_select_next_proto function has been called as expected (with the list
supplied by the client passed in the client/client_len parameters), then the
application will not be vulnerable to this issue. If the application has
accidentally been configured with a zero length server list, and has
accidentally passed that zero length server list in the client/client_len
parameters, and has additionally failed to correctly handle a "no overlap"
response (which would normally result in a handshake failure in ALPN) then it
will be vulnerable to this problem.
In the case of NPN, the protocol permits the client to opportunistically select
a protocol when there is no overlap. OpenSSL returns the first client protocol
in the no overlap case in support of this. The list of client protocols comes
from the application and should never normally be expected to be of zero length.
However if the SSL_select_next_proto function is accidentally called with a
client_len of 0 then an invalid memory pointer will be returned instead. If the
application uses this output as the opportunistic protocol then the loss of
confidentiality will occur.
This issue has been assessed as Low severity because applications are most
likely to be vulnerable if they are using NPN instead of ALPN - but NPN is not
widely used. It also requires an application configuration or programming error.
Finally, this issue would not typically be under attacker control making active
exploitation unlikely.
The FIPS modules in 3.3, 3.2, 3.1 and 3.0 are not affected by this issue.
Due to the low severity of this issue we are not issuing new releases of
OpenSSL at this time. The fix will be included in the next releases when they
become available."""

In [7]:
# sequential chain
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

## prompt for generating attack vector from MITRE MATRIX TTP model
with open("./ttp.md", "r") as f:
    ttp_template = f.read()

ttp_prompt_template = ChatPromptTemplate.from_template(ttp_template)
print(ttp_prompt_template.input_variables)

['company_size', 'industry', 'kill_chain_string', 'selected_group_alias']


In [8]:
# prompt for x-sec to analyze
with open("./x-sec.md", "r") as f:
    xsec_template = f.read()

xsec_prompt_template = ChatPromptTemplate.from_template(xsec_template)
print(xsec_prompt_template.input_variables)

['cve_2024_5535', 'scenario']


In [9]:
from langchain_core.output_parsers import StrOutputParser

# sequencial chain
# chain 1:
#   input=['company_size', 'industry', 'kill_chain_string', 'selected_group_alias']
#   output= ['scenario']
_ttp_chain = LLMChain(llm=llm, prompt=ttp_prompt_template, output_key="scenario")
ttp_chain = (ttp_prompt_template
    | llm
    | StrOutputParser()
)

# chain 2:
#   input= ['scenario']
#   output= summary
_xsec_chain = LLMChain(llm=llm, prompt=xsec_prompt_template, output_key="summary")
xsec_chain = xsec_prompt_template | llm | StrOutputParser()

/var/folders/d4/jnz2593x3yl5s38pz25wxl400000gp/T/ipykernel_74458/2461749673.py:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  _ttp_chain = LLMChain(llm=llm, prompt=ttp_prompt_template, output_key="scenario")


In [11]:
# legacy sequentialchain invoke
from langchain.chains.sequential import SequentialChain

_overall_chain = SequentialChain(
    chains=[_ttp_chain, _xsec_chain],
    input_variables=[
        "company_size",
        "industry",
        "kill_chain_string",
        "selected_group_alias",
        "cve_2024_5535",
    ],
    output_variables=["scenario", "summary"],
    verbose=True,
)
response = _overall_chain(
    {
        "company_size": company_size,
        "industry": industry,
        "kill_chain_string": kill_chain_string,
        "selected_group_alias": selected_group_alias,
        "cve_2024_5535": cve_2024_5535,
    }
)
from IPython.display import display, Markdown

display(Markdown(response["summary"]))

/var/folders/d4/jnz2593x3yl5s38pz25wxl400000gp/T/ipykernel_74458/3567476930.py:16: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = _overall_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.


### Kill Chain of the Given Incident Scenario

The kill chain outlined in the incident scenario follows the MITRE ATT&CK framework and includes the following phases with associated techniques:

1. **Initial Access (T1190):** Exploitation of a public-facing application vulnerability.
2. **Execution (T1053.005):** Scheduled Task established by the attacker.
3. **Persistence (T1505.003):** Deployment of a Web Shell.
4. **Privilege Escalation (Implicit throughout):** Use of techniques like Make and Impersonate Token (T1134.003).
5. **Defense Evasion (T1134.003, T1090.001):** Token manipulation and use of internal proxy for C2.
6. **Credential Access (T1003.001):** Extraction of credentials from LSASS memory.
7. **Discovery (T1087.002):** Enumeration of domain accounts.
8. **Lateral Movement (T1021.006):** Utilizing Windows Remote Management.
9. **Collection (T1005):** Gathering sensitive data from local systems.
10. **Command and Control (T1090.001):** Use of an internal proxy for maintaining C2.
11. **Exfiltration (Implicit):** Not explicitly mentioned but implied with Collection.
12. **Impact (T1657):** Financial theft.

### Open Source Software & Vulnerabilities Related to the Scenario

Given the techniques used by FIN13, relevant open-source software could include:

- **Apache HTTP Server**: Often targeted for exploiting public-facing applications (T1190).
- **WordPress/Drupal/Joomla**: CMS platforms prone to vulnerabilities exploitable for initial access.
- **OpenSSL**: For secure communication, but vulnerabilities can facilitate exploitation.
- **Apache Tomcat**: Application server potentially susceptible to attacks for initial access or command and control.

Regarding **CVE-2024-5535**:

- **Affected Status:** The scenario does not specifically mention the usage of OpenSSL functions like SSL_select_next_proto or the direct involvement of ALPN/NPN protocols. The incident focuses on a targeted attack through a public-facing application, privilege escalation, and data exfiltration rather than vulnerabilities in SSL/TLS negotiation. Therefore, it seems **unlikely** that CVE-2024-5535 directly affects this incident scenario unless the exploited public-facing application relies on a custom implementation of these protocols, which is not typical.

### CVSS Score for CVE-2024-5535

Based on the provided impact summary and considering the scope, the CVSS v3 base score calculation could be approximated as follows:

- **Confidentiality Impact (C):** High, as up to 255 bytes of arbitrary private data could be disclosed.
- **Integrity Impact (I):** None, since the issue does not alter data.
- **Availability Impact (A):** None, as it does not cause service disruption.
- **Attack Vector (AV):** Network (N), if an attacker could manipulate the situation to trigger the bug remotely.
- **Attack Complexity (AC):** High (H), due to the specific conditions required (programming error + zero-length list).
- **Privileges Required (PR):** None (N), assuming the attacker can influence the application’s behavior.
- **User Interaction (UI):**: None (N), as the vulnerability can be triggered without user action.
- **Scope (S):** Unchanged (U), since the vulnerability does not change the security context.

**CVSS v3 Base Score Calculation:**

```
CVSS:3.1/AV:N/AC:H/PR:N/UI:N/S:U/C:H/I:N/A:N
```

Using a CVSS calculator, this results in a score around **6.5**, placing it in the Medium severity range. However, due to the low likelihood of exploitation under normal circumstances and the specific conditions needed, the overall risk might be perceived as lower in practical terms.

In [14]:
#from langchain.callbacks.tracers import ConsoleCallbackHandler

overall_chain = {
    "scenario": ttp_chain,
    "cve_2024_5535": lambda x: cve_2024_5535,
} | xsec_chain
response = overall_chain.invoke(
    {
        "company_size": company_size,
        "industry": industry,
        "kill_chain_string": kill_chain_string,
        "selected_group_alias": selected_group_alias,
        "cve_2024_5535": cve_2024_5535,
    },
    #config={"callbacks": [ConsoleCallbackHandler()]}, # configure for verbose logging
)

from IPython.display import display, Markdown
display(Markdown(response))

### Kill Chain of the Given Incident Scenario

The kill chain for the incident scenario outlined follows the MITRE ATT&CK framework and includes the techniques utilized by the FIN13 threat actor group:

1. **Initial Access (T1190):** Exploit Public-Facing Application
2. **Execution (T1053.005):** Scheduled Task
3. **Persistence (T1505.003):** Web Shell
4. **Privilege Escalation (Implicit through techniques):** Make and Impersonate Token (T1134.003)
5. **Defense Evasion (T1134.003):** Make and Impersonate Token
6. **Credential Access (T1003.001):** LSASS Memory
7. **Discovery (T1087.002):** Domain Account
8. **Lateral Movement (T1021.006):** Windows Remote Management
9. **Collection (T1005):** Data from Local System
10. **Command and Control (C2) (T1090.001):** Internal Proxy
11. **Impact (T1657):** Financial Theft

### Open Source Software Related to the Incident Scenario and Vulnerabilities

Given the scenario focuses on a targeted cyberattack, some open-source software that could be involved, based on the techniques used, include:

1. **Apache HTTP Server**: A commonly used public-facing application that might have vulnerabilities exploitable for initial access (e.g., CVE-2021-41773, a path traversal vulnerability).
2. **WordPress**: A popular CMS often integrated with Apache or other web servers, which could contain vulnerabilities for exploitation (e.g., CVE-2022-2444, a cross-site scripting vulnerability).
3. **OpenSSL**: As indicated by your query about CVE-2024-5535, vulnerabilities in OpenSSL could affect secure communication and potentially be leveraged if the application relies on it.

#### Assessment of CVE-2024-5535 in the Incident

**Affected Status:** Based on the provided scenario, the direct exploitation of CVE-2024-5535 does not seem to align with the tactics, techniques, and procedures (TTPs) of the FIN13 threat actor group. The vulnerability involves a buffer overread in the SSL/TLS protocol handling within OpenSSL, primarily affecting applications that incorrectly use the SSL_select_next_proto function. Since FIN13's TTPs focus on exploiting public-facing applications, maintaining persistence through web shells, and stealing financial data, a vulnerability in SSL/TLS negotiation is less relevant to their primary objectives.

**CVSS Score Analysis:**
The Common Vulnerability Scoring System (CVSS) version 3 provides a standardized method for assessing the severity of vulnerabilities. For CVE-2024-5535:

- **Base Metrics:**
  - **Confidentiality Impact (C):** High (`C:H`) – Up to 255 bytes of arbitrary private data could be leaked.
  - **Integrity Impact (I):** None (`I:N`) – No direct impact on data integrity.
  - **Availability Impact (A):** None (`A:N`) – Does not cause a denial of service.
  - **Attack Vector (AV):** Network (`AV:N`) – Attack can be performed over the network.
  - **Attack Complexity (AC):** Low (`AC:L`) – The attack doesn't require a specific condition to be met.
  - **Privileges Required (PR):** None (`PR:N`) – No privileges required for the attack.
  - **User Interaction (UI):** None (`UI:N`) – Attack doesn't rely on user interaction.
  - **Scope (S):** Unchanged (`S:U`) – The scope of the vulnerability is limited to the impacted component.

Given these metrics, the Base Score calculation would be:
`CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:N/A:N`

However, considering the note that exploitation is unlikely due to application configurations and that NPN is deprecated in favor of ALPN, which is not vulnerable, the environmental and temporal metrics could adjust the overall score.

Based on the provided impact summary and the fact that exploitation depends on several uncommon conditions, the CVSS score might not reach the highest levels. But without the exact CVSS v3 scores, an estimated severity rating based on the description leans towards **Medium** due to the potential for confidentiality loss. 

**Generated Score (Estimate):** 5 out of 10

This estimation assumes a medium severity based on the potential for information disclosure but takes into account the mitigating factors like the low likelihood of occurrence due to specific application misconfigurations needed for exploitation. The actual CVSS score should be calculated with the complete formula and validated against official sources.

In [28]:
# Agent of Google Search Engine
#!pip install --upgrade --quiet  langchain_google_community
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper
from langchain.agents import AgentExecutor, create_react_agent


search = GoogleSearchAPIWrapper()

google_search_tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

# google_search_tool.run("Obama's first name?")
with open("./x-sec-agent.md", 'r') as f:
    agent_prompt = f.read()

agent_prompt_template = ChatPromptTemplate.from_template(agent_prompt)

# Construct the ReAct agent
agent = create_react_agent(llm, tools=[google_search_tool], prompt=agent_prompt_template)

# Prepare scenario info
scenario_prompt = ttp_prompt_template.format_messages(
    company_size = company_size,
    industry=industry,
    kill_chain_string=kill_chain_string,
    selected_group_alias=selected_group_alias,
)
scenario_response = llm.invoke(scenario_prompt)

# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=[google_search_tool], verbose=True)

agent_executor.invoke(
    {
        "input": "what is your favourite food?",
        "scenario": scenario_response.content,
    },
)

agent_executor.invoke(
    {
        "input": "is CVE-2024-5535 affected by the given scenario?",
        "scenario": scenario_response.content,
    },
)

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I can only answer questions about cyber security. 

Final Answer: Sorry, I can only answer questions about cyber security.

> Finished chain.


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


I need to first identify if CVE-2024-5535 is related to any of the TTPs mentioned in the incident response scenario.

Action: google_search
Action Input: "CVE-2024-5535 details"
Jun 27, 2024 ... Issue summary: Calling the OpenSSL API function SSL_select_next_proto with an empty supported client protocols buffer may cause a crash or ... Jun 27, 2024 ... Description. Issue summary: Calling the OpenSSL API function SSL_select_next_proto with an empty supported client protocols buffer may cause a ... CVE-2024-3094 · Description · Statement · Additional information · External references · Affected Packages and Issued Red Hat Security Errata · Common Vulnerability ... Jul 12, 2024 ... 2 are susceptible to a vulnerability which when successfully exploited could lead to disclosure of sensitive information or Denial of Service ( ... Jun 27, 2024 ... Calling the OpenSSL API function SSL_select_next_proto with an empty supported client protocols buffer may cause a crash or memory contents to be s

{'input': 'is CVE-2024-5535 affected by the given scenario?',
 'scenario': "```markdown\n# Incident TTP Scenario\n\n## Overview\nThis incident response testing scenario has been meticulously designed to challenge the company's cybersecurity incident response capabilities against a targeted attack by the FIN13 threat actor group. Leveraging the known tactics, techniques, and procedures (TTPs) of FIN13, this scenario will assess the readiness and effectiveness of the company's defenses and response mechanisms.\n\n## Company Profile\n- **Industry:** Technology / IT\n- **Size:** Large Enterprise (10,000+ employees)\n\n## Threat Actor Information\n- **Threat Actor Group:** FIN13\n- **Kill Chain Phase & Techniques:**\n  - **Initial Access:** Exploit Public-Facing Application (T1190)\n  - **Execution:** Scheduled Task (T1053.005)\n  - **Persistence:** Web Shell (T1505.003)\n  - **Privilege Escalation:** Make and Impersonate Token (T1134.003)\n  - **Defense Evasion:** Make and Impersonate Toke